* Order of execution of SQL Queries
* Generating Explain Plan for SQL Queries
* Interpreting Explain Plan of SQL Queries
* Overview of Indexes on Database Tables
* Different Index Scans based on Queries
* Generate Explain Plans for Application Queries
* Exercise and Solution - Generate Explain Plans

* Order of execution of SQL Queries

Typically we write queries in this order. We are taking simple SQL query statements only.
1. `SELECT` (with columns or derived columns (using expressions or functions)).
2. `FROM` with table names
3. `WHERE` with filter conditions
4. `ORDER BY` with sort criteria

The order of execution is different from order of writing the queries.
1. `FROM` - Reads the data from the files associated with tables into memory.
2. `WHERE` - Filters the data based on conditions using where clause (in the memory).
3. `SELECT` - Filters and apply's the transformations based on columns and expressions in SELECT clause.
4. `ORDER BY` -  sorts the data based on the specified sorting criteria.

Here is the example query.

```sql
SELECT * FROM users
WHERE email like 'sjo%'
ORDER BY id;
```

* Generating Explain Plan for SQL Queries

Explain Plans can be generated using CLIs such as `psql` or IDEs such as `pgAdmin` for Postgres. We will use `pgAdmin`.

1. Develop a query for which explain plan needs to be generated.

```sql
SELECT * FROM users
WHERE id = 1;
```

2. Use `EXPLAIN` command to generate explain plan.

```sql
-- Example for index scan (id is primary key and hence automatically indexed)
EXPLAIN
SELECT * FROM users
WHERE id = 1;

-- Example for full table scan (first_name is not indexed)
EXPLAIN
SELECT * FROM users
WHERE first_name LIKE 'Joh%';
```

* Interpreting Explain Plan of SQL Queries

```text
root
  - branch 1
    - leaf 1
    - leaf 2
  - branch 2
    - leaf 3
    - leaf 4
```

1. Explain plans follow tree structure.
2. You will have root, branches, and then low level leaves.
3. The leaf level will be executed first. If there are multiple leaves at same level then the execution happens in the order of leaves.
4. For now primarily focus on full table scan vs. index scan.

* Overview of Indexes on Database Tables

1. Indexes are typically created on the columns in a table which are typically used for filtering (search).
2. Primary Key Columns and Unique Constraint Columns are always indexed.
3. Data in indexes are sorted. Index also have pointers to corresponding records in the table.

* Different Index Scans based on Queries

1. Index Unique Scan (`username = 'smartinez'`)
2. Index Range Scan (`username LIKE 'sm%'`)
3. Index Full Scan (`username LIKE '%sm%'`)
4. Index Only Scan (when select clause have only columns in the index used)

* Generate Explain Plans for Application Queries

Here are the steps to be followed to generate explain plans for the queries used by Flask Applications.

1. Run application by enabling `SQLALCHEMY_ECHO`.
2. Start the application or launch the Flask Shell.
3. Access the endpoints or use APIs on Models in Flask Shell.
4. Get the queries and values.
5. Connect to underlying database and run `EXPLAIN` on the query with values.
6. Review explain plan and identify the bottlenecks.

Use below code to generate queries.

```python
# Invoking APIs on models
User.query.get(1)

# Invoking REST APIs using end points
from werkzeug.test import Client
client = Client(app)

client.get('/user?id=1')
```

* Exercise - Generate Explain Plans

Generate and Review Explain Plans for the following API calls as well as queries.

1. Make sure to setup retail_db tables in sales_db using pgAdmin.
2. Run the following API call using Flask Shell and generate explain plans. Identify the bottleneck based on the explain plan generated and suggest an index.

```python
User.query.find(User.email.like('sjo%'))
```

3. Run the following queries using pgAdmin. Identify the bottlenecks based on the explain plans generated.

```sql
SELECT count(*)
FROM orders
WHERE order_date = '2014-01-01 00:00:00.0';

SELECT count(*)
FROM orders
WHERE order_date = '2014-01-01 00:00:00.0'
AND order_status = 'COMPLETE';

SELECT count(*)
FROM orders
WHERE order_date = '2014-01-01 00:00:00.0'
AND order_status = 'CLOSED';
```